## Context

ABC Supermarket is planning for the year-end sale - they want to launch a new offer i.e. gold membership for only \\$499 that is of \\$999 on normal days (that gives 20% discount on all purchases) only for existing customers, for that they need to do a campaign through phone calls - the best way to reduce the cost of the campaign is to make a predictive model to classify customers who might purchase the offer, using the data they gathered during last year's campaign.

We will build a model for classifying whether customers will reply with a positive
response or not.

## Import Statements

In [2828]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel("marketing_data.xlsx")

## Column Definitions

- Response (target) - 1 if customer accepted the offer in the last campaign, 0
otherwise
- Complain - 1 if a customer complained in the last 2 years
- DtCustomer - date of customer’s enrolment with the company
- Education - customer’s level of education
- Marital - customer’s marital status
- Kidhome - number of small children in customer’s household
- Teenhome - number of teenagers in customer’s household
- Income - customer’s yearly household income
- MntFishProducts - the amount spent on fish products in the last 2 years
- MntMeatProducts - the amount spent on meat products in the last 2 years
- MntFruits - the amount spent on fruits products in the last 2 years
- MntSweetProducts - amount spent on sweet products in the last 2 years
- MntWines - the amount spent on wine products in the last 2 years
- MntGoldProds - the amount spent on gold products in the last 2 years
- NumDealsPurchases - number of purchases made with discount
- NumCatalogPurchases - number of purchases made using catalog
- NumStorePurchases - number of purchases made directly in stores
- NumWebPurchases - number of purchases made through the company’s web site
- NumWebVisitsMonth - number of visits to company’s web site in the last month
- Recency - number of days since the last purchase
- ID - unique customer-id
- Year_Birth - customer's year of birth

## Data Profiling & Preprocessing

In [ ]:
# df.dtypes
# df.isnull().sum()


In [ ]:
# dfdupes = df
df = df.drop(columns=['ID'])
df = df.drop_duplicates(keep='last')

In [ ]:
# Extracting Year from customer's date of enrolment

from datetime import datetime

df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%m/%d/%y')
df['Dt_Customer'] = df['Dt_Customer'].dt.year

In [ ]:
# Removing Customer Records with Year_Birth earlier than 1940
def remove_centennial_cust(df):
    return df[df['Year_Birth']>=1940]

df = remove_centennial_cust(df)

In [ ]:
# Change Alone, Absurd, YOLO to Single
df['Marital_Status'] = df['Marital_Status'].replace('Alone', 'Single')
df['Marital_Status'] = df['Marital_Status'].replace('Absurd', 'Single')
df['Marital_Status'] = df['Marital_Status'].replace('YOLO', 'Single')

In [ ]:
# Populating Missing Income with Median with Similar Education and MaritalStatus
# median = df.groupby(['Education', 'Marital_Status'])['Income'].transform('median')
# df['Income'] = df['Income'].fillna(median);
# Changed to fill 0
df['Income'] = df['Income'].fillna(0)

In [ ]:
# plt.figure(figsize=(8, 6))
# plt.hist(df['Income'], bins=30, density=True, alpha=0.7, color='blue')
# plt.title('Histogram of Income')
# plt.xlabel('Income')
# plt.ylabel('Density')

In [ ]:
# Convert Year_Birth to Age
df['Age'] = 2015 - df['Year_Birth']
df.drop(columns=['Year_Birth'],inplace=True)

In [ ]:
# Dropping Outliers in Income using STD method
income = df['Income'].values
mean = np.mean(income)
std = np.std(income)

threshold_lower = mean - 2 * std
threshold_upper = mean + 2 * std

df = df[(df['Income'] >= threshold_lower) & (df['Income'] <= threshold_upper)]

In [ ]:
# Dropping Outliers in Income using IQR Method
# def remove_outliers(df, threshold=1.5):
#     Q1 = df['Income'].quantile(0.25)
#     Q3 = df['Income'].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - threshold * IQR
#     upper_bound = Q3 + threshold * IQR
#     return df[(df['Income'] > lower_bound) & (df['Income'] < upper_bound)]


# df = remove_outliers(df)

In [ ]:
# plt.figure(figsize=(8, 6))
# plt.hist(df['Income'], bins=30, density=True, alpha=0.7, color='blue')
# plt.title('Histogram of Income')
# plt.xlabel('Income')
# plt.ylabel('Density')

In [ ]:
# Ratio of Mnt column to the total of Mnt Column; Assuming that the formulas are correct, no effect

# df['MntTotal'] = df[['MntWines','MntFruits','MntMeatProducts','MntFishProducts','MntSweetProducts','MntGoldProds']].sum(axis=1)
# sum = df['MntTotal'].sum()
# aveSum = sum/ len(df)
# df['Ratio'] = df['MntTotal']/aveSum
# df.drop(columns=['MntTotal'],inplace=True)

In [ ]:
# WebTransactions
df['WebTransactions'] = df['NumWebPurchases']**2/df['NumWebVisitsMonth']
df.replace([np.inf,-np.inf, np.nan],0, inplace=True)

In [ ]:
# Total # of Children and Identify if Single Parent
# No effect if no other column is removed
# df['TotalChildren'] = df['Kidhome']+df['Teenhome']
# df['SingleParent'] = False

# # Single
# SingleParentMask = (df['Marital_Status'] == 'Single') & (df['TotalChildren']>0)
# df.loc[SingleParentMask, 'SingleParent'] = True

# # Widow
# SingleParentMask = (df['Marital_Status'] == 'Widow') & (df['TotalChildren']>0)
# df.loc[SingleParentMask, 'SingleParent'] = True

# # Incentivize DealsPurchases if SingleParent, No Effect
# df.loc[df['SingleParent'], 'NumDealsPurchases'] **=2

In [ ]:
## Educ Coding
# educMap = {'Basic':1, '2n Cycle':2, 'Graduation':3,'Master':4,'PhD':5}
# df['Education'] = df['Education'].replace(educMap)

In [ ]:
## If Married with kids No effect
# df['Parents']=False
# df.loc[((df['Marital_Status'].isin(['Married', 'Together'])) & df['TotalChildren']>0),'Parents']=True

In [ ]:
# Income to MNT total No Effect
# df['BoughtItems'] = df['Income']/(df['Mnt+df['MntFruits']+df['MntGoldProds']
#                                            +df['MntMeatFishProducts']Products']+df['MntSweetProducts']+df['MntWines'])

In [ ]:
# No Effect
# bins = [0, 24, 177, 505, 1500]
# labels = [1,2,3,4]

# df['MntWinesNum'] = pd.cut(df['MntWines'],bins=bins, labels=labels, right=False)
# df['MntWinesNum'] = df['MntWinesNum'].astype(int)

In [ ]:
# df['MntWines'].describe()

In [ ]:
## RecencyRelevance
# df['RecencyRelevance'] = df['Recency']**2/(df['MntFishProducts']+df['MntFruits']+df['MntGoldProds']
#                                            +df['MntMeatProducts']+df['MntSweetProducts']+df['MntWines'])

## Initial Exploratory Data Analysis

In [ ]:
# colors = sns.color_palette('pastel')[0:len(df['Response'].value_counts())]

# #create pie chart
# plt.pie(df['Response'].value_counts(), labels=['Didnt Accept', 'Accepted'], colors = colors, autopct='%.0f%%')
# plt.legend()
# plt.show()

In [ ]:
# colors = sns.color_palette('pastel')[0:len(df['Complain'].value_counts())]

# #create pie chart
# plt.pie(df['Complain'].value_counts(), labels=['Didnt Complain', 'Complained'], colors = colors, autopct='%.0f%%')
# plt.legend()
# plt.show()

In [ ]:
# sns.barplot(x = 'Income', y = 'count', data=df[['Income']].value_counts().reset_index(name='count'))
# sns.countplot(x='Income',data=df_complaints)
# plt.xticks(rotation=45)
# plt.show()
# df

In [ ]:
# sns.barplot(x = 'Complain', y='count', hue = 'Response',data=df[['Complain', 'Response']].value_counts().reset_index())
    
# plt.show()

In [ ]:
# fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(12, 4))

# for i, ax, col in zip(range(2), axes.flat, ['Kidhome', 'Teenhome']):
#     tmp = df[[col, 'Response']].value_counts().to_frame().reset_index()
#     tmp['Response'] = tmp['Response'].replace({0: 'Didnt Accept', 1: 'Accepted'})

#     sns.barplot(x = col, y='count', 
#                    hue = 'Response',data=tmp, ax=ax)
    
# plt.show()

In [ ]:
# fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(12, 10))

# for i, ax, col in zip(range(6), axes.flat, ['MntFishProducts', 'MntMeatProducts', 'MntFruits', 'MntSweetProducts', 'MntWines', 'MntGoldProds']):
#     sns.kdeplot(df, x=col, hue='Response', fill=True, ax=ax)
    
# plt.show()

## Baseline Modelling

In [ ]:
def one_hot_encode(data, column, suffix=''):
    encoded = pd.get_dummies(data[column], drop_first=True, prefix=column+suffix)
    data = data.drop(column, axis = 1)
    data = data.join(encoded)
    return data

In [ ]:
df = one_hot_encode(df, 'Marital_Status')
df = one_hot_encode(df, 'Education')

In [ ]:
df.columns

Index(['Year_Birth', 'Income', 'Kidhome', 'Teenhome', 'Dt_Customer', 'Recency',
       'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
       'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases',
       'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
       'NumWebVisitsMonth', 'Response', 'Complain', 'WebTransactions',
       'Marital_Status_Alone', 'Marital_Status_Divorced',
       'Marital_Status_Married', 'Marital_Status_Single',
       'Marital_Status_Together', 'Marital_Status_Widow',
       'Marital_Status_YOLO', 'Education_Basic', 'Education_Graduation',
       'Education_Master', 'Education_PhD'],
      dtype='object')

In [ ]:
# df = df[['Year_Birth', 'Income', 'Kidhome', 'Teenhome', 'Dt_Customer', 'Recency',
#        'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
#        'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases',
#        'NumCatalogPurchases', 'NumStorePurchases',
#         'Response', 'Complain', 'WebTransactions',
#        'Marital_Status_Alone', 'Marital_Status_Divorced',
#        'Marital_Status_Married', 'Marital_Status_Single',
#        'Marital_Status_Together', 'Marital_Status_Widow',
#        'Marital_Status_YOLO', 'Education_Basic', 'Education_Graduation',
#        'Education_Master', 'Education_PhD']]

In [ ]:
# X = df.drop('Response', axis=1).iloc[:,1:][['MntFishProducts', 'MntMeatProducts', 'MntFruits'
#                                             , 'MntSweetProducts', 'MntWines', 'MntGoldProds'
#                                             ]]
X = df.drop(['Response', 'Complain'], axis=1)
y = df['Response']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
from sklearn.naive_bayes import GaussianNB 
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
# from sklearn.

In [ ]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)


In [ ]:
# Table for Model Comparison
# classifiers = {
#     'Logistic Regression': LogisticRegression(),
#     'Gaussian Naive Bayes': GaussianNB(),
#     'Decision Tree': DecisionTreeClassifier(),
#     'SVC': SVC(),
#     'K Nearest Neighbors': KNeighborsClassifier()
# }

# metricsList = []

# for name, clf in classifiers.items():
#     clf.fit(X_train,y_train)

#     preds = clf.predict(scaler.transform(X_test))

#     # Scores
#     acc = accuracy_score(y_test, preds)
#     prec = precision_score(y_test, preds)
#     rec = recall_score(y_test, preds)
#     f1 = f1_score(y_test, preds)
#     auc = roc_auc_score(y_test, preds)
#     trainScore = clf.score(X_train, y_train)
#     testScore = clf.score(X_test, y_test)

#     metrics = {
#         'Classifier': name,
#         'Accuracy': format(acc,'.4f'),
#         'Precision': format(prec,'.4f'),
#         'Recall': format(rec,'.4f'),
#         'F1 Score': format(f1,'.4f'),
#         'AUC': format(auc,'.4f'),
#         'Training': format(trainScore, '.4f'),
#         'Test': format(testScore, '.4f'),
#     }

#     metricsList.append(metrics)

# metrics_df = pd.DataFrame(metricsList)

# metrics_df

In [ ]:
clf = LogisticRegression().fit(X_train, y_train)
preds = clf.predict(scaler.transform(X_test))

acc = accuracy_score(y_test, preds)
prec = precision_score(y_test, preds)
rec = recall_score(y_test, preds)
f1 = f1_score(y_test, preds)
auc = roc_auc_score(y_test, preds)

print("Accuracy: %.4f" % acc)
print("Precision: %.4f" % prec)
print("Recall: %.4f" % rec)
print("F1: %.4f" % f1)
print("AUC: %.4f" % auc)

# Check for overfitting
print('\nTraining set score: {:.4f}'.format(clf.score(X_train, y_train)))
print('Test set score: {:.4f}'.format(clf.score(X_test, y_test)))

Accuracy: 0.8851
Precision: 0.7419
Recall: 0.3710
F1: 0.4946
AUC: 0.6740

Training set score: 0.8667
Test set score: 0.1711


C:\Users\Predator\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


### Result Matrix and Graphs

In [ ]:
# conf_mat = confusion_matrix(y_test, preds)
# plt.figure(figsize=(6, 4))
# sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')
# plt.xlabel('Predicted labels')
# plt.ylabel('True labels')
# plt.title('Confusion Matrix')
# plt.show()

In [ ]:
## Feature Importance
# feature_names = X.columns
# plt.figure(figsize=(10, 6))
# plt.barh(np.arange(len(feature_names)), clf.coef_[0], color='skyblue')
# plt.yticks(np.arange(len(feature_names)), feature_names)
# plt.xlabel('Coefficient Value')
# plt.ylabel('Feature')
# plt.title('Logistic Regression Coefficients')
# plt.grid(True)
# plt.show()